In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()
print(spark.sparkContext.version)

3.3.2


- Union and UnionAll both works same in Spark latest versions. 
- To remove duplicate one needs to use dropDuplicates() explicitly

### Create Dataframe df1

In [0]:
employee_data = [
    (100, 'Stephen', '1999', '100', 'M', 2000),
    (200, 'Philip', '2002', '200', 'M', 8000),
    (300, 'John', '2010', '100', '', 6000)
]
employee_schema = ['employee_id', 'name', 'doj', 'employee_dept_id', 'gender', 'salary']

df1= spark.createDataFrame(employee_data, employee_schema)
display(df1)

employee_id,name,doj,employee_dept_id,gender,salary
100,Stephen,1999,100,M,2000
200,Philip,2002,200,M,8000
300,John,2010,100,,6000


%md
### Create Dataframe df2

In [0]:
employee_data = [
    (300, 'John', '2010', '100', '', 6000),
    (400, 'Nancy', '2008', '400', 'F', 1000),
    (500, 'Rosy', '2014', '500', 'M', 5000)
]
employee_schema = ['employee_id', 'name', 'doj', 'employee_dept_id', 'gender', 'salary']

df2= spark.createDataFrame(employee_data, employee_schema)
display(df2)

employee_id,name,doj,employee_dept_id,gender,salary
300,John,2010,100,,6000
400,Nancy,2008,400,F,1000
500,Rosy,2014,500,M,5000


### Union

In [0]:
df_union = df1.union(df2)
display(df_union)

employee_id,name,doj,employee_dept_id,gender,salary
100,Stephen,1999,100,M,2000
200,Philip,2002,200,M,8000
300,John,2010,100,,6000
300,John,2010,100,,6000
400,Nancy,2008,400,F,1000
500,Rosy,2014,500,M,5000


### Drop Duplicates

In [0]:
display(df_union.dropDuplicates())


employee_id,name,doj,employee_dept_id,gender,salary
100,Stephen,1999,100,M,2000
200,Philip,2002,200,M,8000
300,John,2010,100,,6000
400,Nancy,2008,400,F,1000
500,Rosy,2014,500,M,5000


### UnionAll

In [0]:
df_unionall = df1.unionAll(df2)
display(df_unionall)

employee_id,name,doj,employee_dept_id,gender,salary
100,Stephen,1999,100,M,2000
200,Philip,2002,200,M,8000
300,John,2010,100,,6000
300,John,2010,100,,6000
400,Nancy,2008,400,F,1000
500,Rosy,2014,500,M,5000


### Schema Mismatch

In [0]:
df3 = df2.select(df2.employee_id, df2.name, df2.doj, df2.employee_dept_id, df2.gender) # excluded salary intentionally
display(df3)

employee_id,name,doj,employee_dept_id,gender
300,John,2010,100,
400,Nancy,2008,400,F
500,Rosy,2014,500,M


In [0]:
df4= df1.union(df3)
display(df4)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5378807170545673>:1
----> 1 df4= df1.union(df3)
      2 display(df4)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3646, in DataFrame.union(self, other)
   3598 def union(self, other: "DataFrame") -> "DataFrame":
   3599     """Return a new :class:`DataFrame` containing union of rows in this and another
   3600     :class:`DataFrame`.
   3601 
   (...)
   3644     +----+----+----+
   3645     """
-> 3646     return DataFrame(self._jdf.union(other.